In [1]:
from __future__ import division
import pandas as pd
import pickle as cPickle
import numpy as np
import scipy.io as sio
import scipy.sparse as ss
from numpy.random import random  
from collections import defaultdict

import warnings
warnings.filterwarnings('ignore')

###  推荐系统类

In [24]:
class RecommonderSystem:
  def __init__(self):
    # 读入数据做初始化
    
    #用户和活动新的索引
    self.userIndex = cPickle.load(open("Data/PE_userIndex.pkl", 'rb'))
    self.eventIndex = cPickle.load(open("Data/PE_eventIndex.pkl", 'rb'))
    self.n_users = len(self.userIndex)
    self.n_items = len(self.eventIndex)
    
    #用户-活动关系矩阵R
    self.userEventScores = sio.mmread("Data/PE_userEventScores").todense()
    
    #倒排表
    ##每个用户参加的事件
    self.itemsForUser = cPickle.load(open("Data/PE_eventsForUser.pkl", 'rb'))
    ##事件参加的用户
    self.usersForItem = cPickle.load(open("Data/PE_usersForEvent.pkl", 'rb'))
    
    #基于模型的协同过滤参数初始化,训练
    self.init_SVD()
    self.train_SVD(trainfile = "Data/train.csv")
    
    #根据用户属性计算出的用户之间的相似度
    self.userSimMatrix = sio.mmread("Data/US_userSimMatrix").todense()
    
    #根据活动属性计算出的活动之间的相似度
    self.eventPropSim = sio.mmread("Data/EV_eventPropSim").todense()
    self.eventContSim = sio.mmread("Data/EV_eventContSim").todense()
    
    #每个用户的朋友的数目
    self.numFriends = sio.mmread("Data/UF_numFriends")
    #用户的每个朋友参加活动的分数对该用户的影响
    self.userFriends = sio.mmread("Data/UF_userFriends").todense()
    
    #活动本身的热度
    self.eventPopularity = sio.mmread("Data/EA_eventPopularity").todense()

  def init_SVD(self, K=20):
    #初始化模型参数（for 基于模型的协同过滤SVD_CF）
    self.K = K  
    
    #init parameters
    #bias
    self.bi = np.zeros(self.n_items)  
    self.bu = np.zeros(self.n_users)  
    
    #the small matrix
    self.P = random((self.n_users,self.K))/10*(np.sqrt(self.K))
    self.Q = random((self.K, self.n_items))/10*(np.sqrt(self.K))  
                  
          
  def train_SVD(self,trainfile = 'Data/train.csv', steps=100,gamma=0.04,Lambda=0.15):
    #训练SVD模型（for 基于模型的协同过滤SVD_CF）
    #gamma：为学习率
    #Lambda：正则参数
    
    print ("SVD Train...")
    ftrain = open(trainfile, 'r')
    ftrain.readline()
    self.mu = 0.0
    n_records = 0
    uids = []  #每条记录的用户索引
    i_ids = [] #每条记录的item索引
    #用户-Item关系矩阵R（内容同userEventScores相同），临时变量，训练完了R不再需要
    R = np.zeros((self.n_users, self.n_items))
    
    for line in ftrain:
        cols = line.strip().split(",")
        u = self.userIndex[cols[0]]  #用户
        i = self.eventIndex[cols[1]] #活动
        
        uids.append(u)
        i_ids.append(i)
        
        R[u,i] = int(cols[4])  #interested
        self.mu += R[u,i] # mu 是整个评分的一个平均数
        n_records += 1
    
    ftrain.close()
    self.mu /= n_records
    
    # code begin 
    
    for step in range(steps):  
        rmse_sum=0.0 

        # 随机打乱样本顺序
        kk = np.random.permutation(n_records)  
        for j in range(n_records):  
            
            u = uids[kk[j]]
            i = i_ids[kk[j]]
            # 残差计算    
            re_ui = R[u,i] - self.pred_SVD(u,i) - self.bu[u] - self.bi[i]- np.dot(self.P[u,:],self.Q[:,i])
            re_ui= 1 if re_ui>1 else 0
            # 残差平方和
            rmse_sum += re_ui **2
                            
            #SGD
            self.bu[u] += gamma * (re_ui - Lambda * self.bu[u])  
            self.bi[i] += gamma * (re_ui - Lambda * self.bi[i]) 
            # 更新 P Q
            for k in range(self.K):
                self.P[u,k] += gamma * re_ui * self.Q[k,i] - Lambda * self.P[u,k]
                self.Q[k,i] += gamma * re_ui * self.P[u,k] - Lambda * self.Q[k,i]
        
        rmse_sum = 0.5 * rmse_sum
        
        pmatrix_sum = 0.5 * Lambda * self.P[u,:].dot(self.P[u,:].T)
        qmatrix_sum = 0.5 * Lambda * self.Q[:,i].dot(self.Q[:,i].T)
        
        # 偏置
        b_u_sum = 0.5 * Lambda* (self.bu**2).sum()
        b_i_sum =0.5 * Lambda* (self.bi**2).sum()

        # 目标函数
        sse = rmse_sum + ( pmatrix_sum + qmatrix_sum )+ (b_u_sum + b_i_sum)
        if sse < 0.001:
            print ("the sse is" +sse)
        break
        
        delta = 0.95 
        # 学习率递减
        gamma = gamma*delta  
    
    # code end
    
    # 请补充完整SVD模型训练过程
    print ("SVD trained")
    
  def pred_SVD(self, uid, i_id):
    #根据当前参数，预测用户uid对Item（i_id）的打分        
    ans=self.mu + self.bi[i_id] + self.bu[uid] + np.dot(self.P[uid,:],self.Q[:,i_id])  
        
    #将打分范围控制在0-1之间
    if ans>1:  
        return 1  
    elif ans<0:  
        return 0
    return ans  

  def sim_cal_UserCF(self, uid1, uid2 ):
    #请补充基于用户的协同过滤中的两个用户uid1和uid2之间的相似度
    
    # code begin
    both_i = {}  # 两个用户均有打分的item
    
    for item in self.itemsForUser[uid1]:  # uid1所有打过分的item1
        if item in self.itemsForUser[uid2]:  # 如果uid2也对该item打过分
            both_i[item]=1  
        
    n= len(both_i)   
    if (n==0):  
        return 0  
        
    # uid1打过分的有效item
    s1= np.array([self.userEventScores[uid1,item] for item in both_i])  
        
    # uid2打过分的有效item
    s2= np.array([self.userEventScores[uid2,item] for item in both_i])  
        
    sum1= np.sum(s1)  
    sum2= np.sum(s2)  
    sum1Sq= np.sum(s1**2)  
    sum2Sq= np.sum(s2**2)  
    pSum= np.sum(s1*s2)  
        
    num= pSum-(sum1*sum2 / n)  
        
    den= np.sqrt((sum1Sq-sum1**2 / n) * (sum2Sq-sum2**2 / n))  
    if den==0:  
        return 0  
        
    similarity = num/den  
    return similarity   
    # code end
    

  def userCFReco(self, userId, eventId):
    """
    根据User-based协同过滤，得到event的推荐度
    基本的伪代码思路如下：
    for item i
      for every other user v that has a preference for i
        compute similarity s between u and v
        incorporate v's preference for i weighted by s into running average
    return top items ranked by weighted average
    """
    #请补充完整代码
    
    # code begin
    
    u = self.userIndex[userId]
    i = self.eventIndex[eventId]
    
    sim_accumulate=0.0  
    rat_acc=0.0  
    
    for user in self.usersForItem[i]:  #对eventId打过分的所有用户
        
        sim = self.sim_cal_UserCF(uid1 = user,uid2 = u)    #该user与uid之间的相似度
        if sim == 0:continue  
            
        rat_acc += sim * self.userEventScores[user,i]   #用户user对eventId的打分
        
        sim_accumulate += sim  
        
    if sim_accumulate==0: 
        return  self.mu  
    ans = rat_acc/sim_accumulate  

    #打分控制在0-1间
    if ans>1:  
        return 1  
    elif ans<0:  
        return 0  
    return ans

    # code end
    

  def sim_cal_ItemCF(self, i_id1, i_id2):
    #计算Item i_id1和i_id2之间的相似性
    
    # code begin
    
    both_u= {}  # 有效用户集合
    for user in self.usersForItem[i_id1]:  # 所有对item1打过分的的user
        if user in self.usersForItem[i_id2]:  # 如果该用户对item2也打过分
            both_u[user]=1  # 有效用用户
        
    n= len(both_u)  
    if (n==0):  
        return 0  
        
    s1= np.array([self.userEventScores[u, i_id1] for u in both_u])  
        
    s2= np.array([self.userEventScores[u, i_id2] for u in both_u])  
        
    sum1= np.sum(s1)  
    sum2= np.sum(s2)  
    sum1Sq= np.sum(s1**2)  
    sum2Sq= np.sum(s2**2)  
    pSum= np.sum(s1*s2)  
        
    num= pSum-(sum1*sum2/n)  
        
    den= np.sqrt((sum1Sq-sum1**2 / n) * (sum2Sq-sum2**2 / n))  
    if den==0:  
        return 0  
        
    return num/den 
    
    # code end

            
  def eventCFReco(self, userId, eventId):    
    """
    根据基于物品的协同过滤，得到Event的推荐度
    基本的伪代码思路如下：
    for item i 
        for every item j tht u has a preference for
            compute similarity s between i and j
            add u's preference for j weighted by s to a running average
    return top items, ranked by weighted average
    """
    #请补充完整代码
    ans = 0.0
    
    # code begin
    
    u = self.userIndex[userId]
    i = self.eventIndex[eventId]

    sim_accumulate=0.0  
    rat_acc=0.0  
                   
    for item in self.itemsForUser[u]:  # 用户uid打过分的所有item
        sim = self.sim_cal_ItemCF(item,i)    #该item与i之间的相似度
           
        rat_acc += sim * self.userEventScores[u,item]  
        sim_accumulate += sim  
        
    if sim_accumulate==0: 
        return  self.mu  

    ans = rat_acc/sim_accumulate  

    # 打分范围控制在 0-1
    if ans>1:  
        return 1  
    elif ans<0:  
        return 0
    return ans
    
    # code end
    
  def svdCFReco(self, userId, eventId):
    #基于模型的协同过滤, SVD++/LFM
    u = self.userIndex[userId]
    i = self.eventIndex[eventId]

    return self.pred_SVD(u,i)

  def userReco(self, userId, eventId):
    """
    类似基于User-based协同过滤，只是用户之间的相似度由用户本身的属性得到，计算event的推荐度
    基本的伪代码思路如下：
    for item i
      for every other user v that has a preference for i
        compute similarity s between u and v
        incorporate v's preference for i weighted by s into running aversge
    return top items ranked by weighted average
    """
    i = self.userIndex[userId]
    j = self.eventIndex[eventId]

    vs = self.userEventScores[:, j]
    sims = self.userSimMatrix[i, :]

    prod = sims * vs

    try:
      return prod[0, 0] - self.userEventScores[i, j]
    except IndexError:
      return 0

  def eventReco(self, userId, eventId):
    """
    类似基于Item-based协同过滤，只是item之间的相似度由item本身的属性得到，计算Event的推荐度
    基本的伪代码思路如下：
    for item i 
      for every item j that u has a preference for
        compute similarity s between i and j
        add u's preference for j weighted by s to a running average
    return top items, ranked by weighted average
    """
    i = self.userIndex[userId]
    j = self.eventIndex[eventId]
    js = self.userEventScores[i, :]
    psim = self.eventPropSim[:, j]
    csim = self.eventContSim[:, j]
    pprod = js * psim
    cprod = js * csim
    
    pscore = 0
    cscore = 0
    try:
      pscore = pprod[0, 0] - self.userEventScores[i, j]
    except IndexError:
      pass
    try:
      cscore = cprod[0, 0] - self.userEventScores[i, j]
    except IndexError:
      pass
    return pscore, cscore

  def userPop(self, userId):
    """
    基于用户的朋友个数来推断用户的社交程度
    主要的考量是如果用户的朋友非常多，可能会更倾向于参加各种社交活动
    """
    if userId in self.userIndex:
      i = self.userIndex[userId]
      try:
        return self.numFriends[0, i]
      except IndexError:
        return 0
    else:
      return 0

  def friendInfluence(self, userId):
    """
    朋友对用户的影响
    主要考虑用户所有的朋友中，有多少是非常喜欢参加各种社交活动/event的
    用户的朋友圈如果都积极参与各种event，可能会对当前用户有一定的影响
    """
    nusers = np.shape(self.userFriends)[1]
    i = self.userIndex[userId]
    return (self.userFriends[i, :].sum(axis=0) / nusers)[0,0]

  def eventPop(self, eventId):
    """
    本活动本身的热度
    主要是通过参与的人数来界定的
    """
    i = self.eventIndex[eventId]
    return self.eventPopularity[i, 0]


### 生成推荐数据

In [25]:
def generateRSData(RS, train=True, header=True):
    """
    把前面user-based协同过滤 和 item-based协同过滤，以及各种热度和影响度作为特征组合在一起
    生成新的训练数据，用于分类器分类使用
    """
    fn = "train.csv" if train else "test.csv"
    fin = open(fn, 'rb')
    fout = open("RS_" + fn, 'w')
    
    #忽略第一行（列名字）
    fin.readline().decode().strip().split(",")
    
    # write output header
    if header:
      ocolnames = ["invited", "userCF_reco", "evtCF_reco","svdCF_reco","user_reco", "evt_p_reco",
        "evt_c_reco", "user_pop", "frnd_infl", "evt_pop"]
      if train:
        ocolnames.append("interested")
        ocolnames.append("not_interested")
      fout.write(",".join(ocolnames) + "\n")
    
    ln = 0
    for line in fin:
      ln += 1
      if ln%500 == 0:
          print ("%s:%d (userId, eventId)=(%s, %s)" % (fn, ln, userId, eventId))
          #break
      
      cols = line.decode().strip().split(",")
      userId = cols[0]
      eventId = cols[1]
      invited = cols[2]
      
      userCF_reco = RS.userCFReco(userId, eventId)
      itemCF_reco = RS.eventCFReco(userId, eventId)
      svdCF_reco = RS.svdCFReco(userId, eventId)
        
      user_reco = RS.userReco(userId, eventId)
      evt_p_reco, evt_c_reco = RS.eventReco(userId, eventId)
      user_pop = RS.userPop(userId)
     
      frnd_infl = RS.friendInfluence(userId)
      evt_pop = RS.eventPop(eventId)
      ocols = [invited, userCF_reco, itemCF_reco, svdCF_reco,user_reco, evt_p_reco,
        evt_c_reco, user_pop, frnd_infl, evt_pop]
      
      if train:
        ocols.append(cols[4]) # interested
        ocols.append(cols[5]) # not_interested
      fout.write(",".join(map(lambda x: str(x), ocols)) + "\n")
    
    fin.close()
    fout.close()


#### 实例化推荐系统类,生成推荐数据文件

In [26]:
RS = RecommonderSystem()
print ("生成训练数据...\n")
generateRSData(RS,train=True,  header=True)
print ("生成测试数据...\n")
generateRSData(RS, train=False, header=True)

SVD Train...
SVD trained
生成训练数据...

train.csv:500 (userId, eventId)=(123290209, 1887085024)
train.csv:1000 (userId, eventId)=(272886293, 199858305)
train.csv:1500 (userId, eventId)=(395305791, 1582270949)
train.csv:2000 (userId, eventId)=(527523423, 3272728211)
train.csv:2500 (userId, eventId)=(651258472, 792632006)
train.csv:3000 (userId, eventId)=(811791433, 524756826)
train.csv:3500 (userId, eventId)=(985547042, 1269035551)
train.csv:4000 (userId, eventId)=(1107615001, 173949238)
train.csv:4500 (userId, eventId)=(1236336671, 3849306291)
train.csv:5000 (userId, eventId)=(1414301782, 2652356640)
train.csv:5500 (userId, eventId)=(1595465532, 955398943)
train.csv:6000 (userId, eventId)=(1747091728, 2131379889)
train.csv:6500 (userId, eventId)=(1914182220, 955398943)
train.csv:7000 (userId, eventId)=(2071842684, 1076364848)
train.csv:7500 (userId, eventId)=(2217853337, 3051438735)
train.csv:8000 (userId, eventId)=(2338481531, 2525447278)
train.csv:8500 (userId, eventId)=(2489551967, 5206

In [6]:
rs_train = pd.read_csv("RS_train.csv")
rs_train.head(9)

,invited,userCF_reco,evtCF_reco,svdCF_reco,user_reco,evt_p_reco,evt_c_reco,user_pop,frnd_infl,evt_pop,interested,not_interested
0,0,0.268282,0.268282,0.488980,0.000000,1.059746e+00,0.971727,0.000231,0.0,0.000000,0,0
1,0,0.268282,0.268282,0.441393,0.000000,1.820462e-01,0.551649,0.000231,0.0,-0.000012,0,0
2,0,0.268282,0.268282,0.268282,1.667252,-1.000000e+00,-1.000000,0.000231,0.0,-0.000018,1,0
3,0,0.268282,0.268282,0.567770,0.000000,1.199822e+00,0.813238,0.000231,0.0,0.000032,0,0
4,0,0.268282,0.268282,0.268282,0.179932,2.258463e-07,0.977072,0.000231,0.0,0.000039,0,0
5,0,0.268282,0.268282,0.268282,0.000025,2.032554e-07,0.977072,0.000231,0.0,-0.000163,0,0
6,0,0.268282,0.268282,0.337757,0.000000,1.923380e+00,2.989740,0.000160,0.0,-0.000005,0,0
7,0,0.268282,0.268282,0.270455,0.000000,1.927441e-01,0.896462,0.000160,0.0,0.000044,1,0
8,0,0.268282,0.268282,0.270662,0.000000,-4.044681e-01,0.924551,0.000160,0.0,0.000064,1,0


In [7]:
rs_test = pd.read_csv("RS_test.csv")
rs_test.head(9)

,invited,userCF_reco,evtCF_reco,svdCF_reco,user_reco,evt_p_reco,evt_c_reco,user_pop,frnd_infl,evt_pop
0,0,0.268282,0.268282,1.000000,0.0,0.0,0.0,0.000118,0.0,0.000124
1,0,0.268282,0.268282,1.000000,0.0,0.0,0.0,0.000118,0.0,0.000002
2,0,0.268282,0.268282,1.000000,0.0,0.0,0.0,0.000118,0.0,-0.000002
3,0,0.268282,0.268282,1.000000,0.0,0.0,0.0,0.000118,0.0,0.000035
4,0,0.268282,0.268282,0.881089,0.0,0.0,0.0,0.000118,0.0,-0.000030
5,0,0.268282,0.268282,1.000000,0.0,0.0,0.0,0.000118,0.0,0.000253
6,0,0.268282,0.268282,1.000000,0.0,0.0,0.0,0.000118,0.0,-0.000014
7,0,0.268282,0.268282,0.860794,0.0,0.0,0.0,0.000009,0.0,0.000037
8,0,0.268282,0.268282,1.000000,0.0,0.0,0.0,0.000009,0.0,0.000028
